In [4]:
import cv2
import numpy as np
from tqdm.notebook import tqdm
import os

In [5]:
DATASET = "testing_set"

VIDEO_DIR = f"{DATASET}/Videos"
FRAME_DIR = f"{DATASET}/N-Frames"
NUM_FRAMES_PER_VIDEO = 8 #number of frame we want to extract

# Extract n frame per video

In [6]:
def get_total_num_frames(video):
    total = 0
    while True:
        (grabbed, frame) = video.read()
        if not grabbed:
            break
        total += 1
    return total

In [7]:
def get_frame_indexes(num_frames, n):
    return [int(i) for i in np.around(np.linspace(0, num_frames, n))]

In [18]:
def get_frame_iteratively(video_file, frame_index):
    cap = cv2.VideoCapture(video_file)
    extracted = False
    while not extracted:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        extracted, frame = cap.read()
        frame_index -= 1 # try the previous frame
        
    cap.release()
    
    return frame, frame_index

In [19]:
def extract_num_frames(video_filename, num_frames, frame_dir = FRAME_DIR, video_dir = VIDEO_DIR):
    video_name = video_filename.split(".")[0].zfill(5)
    video_path = f"{VIDEO_DIR}/{video_filename}"
    
    cap = cv2.VideoCapture(video_path)

    total_frames = get_total_num_frames(cap)
    frame_indexes = get_frame_indexes(total_frames, num_frames)
    
    frames_folder = f"{frame_dir}/{video_name}"
    if not os.path.exists(frames_folder):
        os.mkdir(frames_folder)

    for frame_index in frame_indexes:
        frame_filename = f"{frames_folder}/{str(frame_index).zfill(3)}.png"
    
        if not os.path.exists(frame_filename): 
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index - 1)
            res, frame = cap.read()
            if res == 0:
                frame, frame_index = get_frame_iteratively(video_path, frame_index)
            else:
                cv2.imwrite(frame_filename, frame)
                
    cap.release()

In [20]:
# Extract NUM_FRAMES_PER_VIDEO frames from each video
if not os.path.exists(FRAME_DIR):
    os.mkdir(FRAME_DIR)
for filename in tqdm(video_filenames, 
                     desc = f"Extracting {NUM_FRAMES_PER_VIDEO} frames per video"):
    extract_num_frames(filename, num_frames = NUM_FRAMES_PER_VIDEO)
    

In [22]:
sorted([2,3,1])

[1, 2, 3]